# verilerin lineer regresyon ile tahmin edilmesi

In [44]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [45]:
df = pd.read_csv('data_cleaned.csv')

In [93]:
#değişken türlerini düzenli bir biçimde tanımladık
df['İlçe'] = df['İlçe'].str.strip()
df['Mahalle'] = df['Mahalle'].str.strip()
df['İlçe'] = df['İlçe'].astype('category')
df['Mahalle'] = df['Mahalle'].astype('category')
df['Oda'] = df['Oda'].astype('int')
df['Salon'] = df['Salon'].astype('int')
df['m^2'] = df['m^2'].astype('int')
df['Bina Yaşı'] = df['Bina Yaşı'].astype('int')
df['Kat'] = df['Kat'].astype('int')
df['Fiyat'] = df['Fiyat'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8434 entries, 0 to 8433
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   İlçe       8434 non-null   category
 1   Mahalle    8434 non-null   category
 2   Oda        8434 non-null   int64   
 3   Salon      8434 non-null   int64   
 4   m^2        8434 non-null   int64   
 5   Bina Yaşı  8434 non-null   int64   
 6   Kat        8434 non-null   int64   
 7   Fiyat      8434 non-null   int64   
dtypes: category(2), int64(6)
memory usage: 431.9 KB


In [47]:
categorical_features = ['İlçe', 'Mahalle']
numerical_features = ['Oda', 'Salon', 'm^2','Bina Yaşı', 'Kat']

In [94]:
#veri ön işleme pipeline'ı oluşuturuluyor. amaç hem sayısal hem de kategorik değişkenlere farklı dönüşün işlemleri uygulamak
#column transformer'ın amacı farklı sütun türlerine (nümerik ve kategorik), birden fazla dönüşüm uygulamak için
#sayısal değerleri 1 ve 0 arasında standardize ediyoruz. 
#kategorik sütunlara one hot encoding uygulanır, handle unknown ise eğitim sırasında karşılaşılmayan veri testte karşımıza çıkarsa hata almamak içind
full_pipeline = ColumnTransformer([
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

In [96]:
#makine öğrenimi algoritması olarak kullanacağımız lineer regresyon'da bağımlı ve bağımsız değişkenleri atadık
X = df.drop('Fiyat', axis=1)
y = df['Fiyat']



In [97]:
#veriyi eğitim ve test setlerine böldük
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [98]:
#pipeline, veri hazırlığı ve modelleme işlemlerini bir araya getirir.
#amacı veri ön işleme ve modelleme adımlarını zincirlemek
#birinci adımda, oluşturmuş olduğumuz full_pipeline ile veri ön işleme yapılır
#sayısal sütunlar standardize edilir, kategorik sütunlar one hot encoding e dönüştürülür
#modeli lineer regresyon ile kuracağımızı parametre olarak girdik
#özetle pipeline, preparation ve model işlemlerinin özellik ve parametrelerini aldı, model için bir boru hattı oluşturdu,modelin değişkenleri ve değerleri atanıp model adlı bir değişkene atandı
model = Pipeline([
    ('preparation', full_pipeline),
    ('model', LinearRegression())
])

In [99]:
model.fit(X_train, y_train)

Pipeline(steps=[('preparation',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Oda', 'Salon', 'm^2',
                                                   'Bina Yaşı', 'Kat']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['İlçe', 'Mahalle'])])),
                ('model', LinearRegression())])

In [100]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

In [102]:
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R^2: {r2}")

MSE: 18688054.753488723
RMSE: 4322.968280416677
R^2: 0.6418018656517213


In [103]:
feature_importances = model.named_steps['model'].coef_
print(len(feature_importances))
print(feature_importances)

391
[-1.40661144e+02  0.00000000e+00  2.66907820e+03 -2.61978072e+03
  8.11332782e+02 -5.18237173e+03  1.56938799e+02 -2.78615844e+03
  4.66207743e+01 -4.41503672e+03  4.33779124e+02  3.84137947e+03
 -4.02561529e+03 -1.90218075e+03  4.62769620e+03  1.84787919e+03
  8.71085103e+02 -6.04720403e+03  1.27788735e+03  1.47271438e+03
  3.43751839e+03  1.16496139e+04 -5.30454570e+03  1.67539089e+03
 -2.57556840e+03 -2.69060909e+03  2.16238914e+03 -3.05298903e+03
  7.79282944e+02  3.73652921e+03 -2.04589024e+03 -1.07641067e+04
  6.96140907e+02  9.44493619e+03 -1.31088164e+03 -1.84868613e+02
 -5.62552947e+03 -4.21070687e+03 -1.00114059e+03  6.20838926e+02
 -1.28321700e+04 -7.26216455e+01  1.14904161e+04 -3.16714417e+03
  6.27056757e+02 -4.04916620e+03 -4.07151710e+03 -1.88136493e+03
  3.16877846e+03 -1.03334060e+04 -6.89779000e+03 -1.95158839e+03
  2.44969318e+03 -4.55960398e+03 -1.03374415e+03 -3.73508919e+03
  2.42731365e+03 -2.38259600e+03 -4.02738822e+03  5.07094414e+03
  3.67341647e+03  1.6

In [105]:
print("Numerical Features")
for i in range(len(numerical_features)):
    print(numerical_features[i], feature_importances[i])

Numerical Features
Oda -140.6611440651727
Salon 0.0
m^2 2669.0781967919775
Bina Yaşı -2619.7807152707705
Kat 811.3327816696072


In [107]:
print("Categorical Features")
for i in range(len(categorical_features)):
    for j in range(len(model.named_steps['preparation'].transformers_[1][1].categories_[i])):
        print(model.named_steps['preparation'].transformers_[1][1].categories_[i][j], feature_importances[len(numerical_features) + j])

Categorical Features
Akyurt -5182.371733946933
Altındağ 156.93879852638787
Ayaş -2786.158440184677
Beypazarı 46.62077431307669
Elmadağ -4415.036721034778
Etimesgut 433.7791239516528
Gölbaşı 3841.379471270377
Haymana -4025.6152896582394
Kahramankazan -1902.1807450489032
Keçiören 4627.6962046836325
Kızılcahamam 1847.8791947880209
Mamak 871.0851031855319
Polatlı -6047.204030462396
Pursaklar 1277.8873471696309
Sincan 1472.7143793870239
Yenimahalle 3437.5183867710625
Çankaya 11649.613878573666
Çubuk -5304.545702311924
100. Yıl -5182.371733946933
19 Mayıs 156.93879852638787
23 Nisan -2786.158440184677
25 Mart 46.62077431307669
29 Ekim -4415.036721034778
30 Ağustos 433.7791239516528
Abidinpaşa 3841.379471270377
Adnan Menderes -4025.6152896582394
Ahi Evran -1902.1807450489032
Ahi Evran OSB 4627.6962046836325
Ahi Mesut 1847.8791947880209
Ahlatlıbel 871.0851031855319
Akdere -6047.204030462396
Akpınar 1277.8873471696309
Aktepe 1472.7143793870239
Akşemsettin 3437.5183867710625
Alacaatlı 11649.6138

In [114]:
new_data = pd.DataFrame({
    'İlçe': ['Gölbaşı'],
    'Mahalle': ['Bahçelievler'],
    'Oda': [2],
    'Salon': [1],
    'm^2': [90],
    'Bina Yaşı': [10],
    'Kat': [-1]
})

print(model.predict(new_data))

[21683.37703354]


In [121]:
print(df[(df['İlçe'] == 'Gölbaşı') & (df['Mahalle'] == 'Bahçelievler') ])


         İlçe       Mahalle  Oda  Salon  m^2  Bina Yaşı  Kat  Fiyat
542   Gölbaşı  Bahçelievler    3      1  160          5    2  25000
543   Gölbaşı  Bahçelievler    3      1  160          5    2  25000
749   Gölbaşı  Bahçelievler    3      1  165         11    3  29500
1238  Gölbaşı  Bahçelievler    3      1  142         20    3  16000
1728  Gölbaşı  Bahçelievler    1      1   55         20    2  14500
2030  Gölbaşı  Bahçelievler    1      1   60          0    3  27000
2185  Gölbaşı  Bahçelievler    3      1  130         16    3  19000
2289  Gölbaşı  Bahçelievler    3      1  110         11    1  19500
2543  Gölbaşı  Bahçelievler    3      1  115         10    1  22500
2707  Gölbaşı  Bahçelievler    1      1   65         15    4  20000
2831  Gölbaşı  Bahçelievler    3      1  130         16    4  21000
3094  Gölbaşı  Bahçelievler    3      1  120         11    1  21000
3817  Gölbaşı  Bahçelievler    1      1   55          0    3  22000
4062  Gölbaşı  Bahçelievler    3      1  130    